# Here's sample usages for pandas-datareader

See [here](https://oeconomicus.jp/2021/05/python-economicdata/) for more usages.


- World Bank
  - https://pandas-datareader.readthedocs.io/en/latest/remote_data.html#world-bank
  - https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-api-country-queries

- Econdb
  - https://www.econdb.com/main-indicators


In [2]:
from pandas_datareader import wb

# look for world bank data
matches = wb.search('employee')[['id', 'name', 'sourceNote', 'sourceOrganization']]
matches

,id,name,sourceNote,sourceOrganization
935,9.0.Employee.All,Employees (%),Share of the labor force (ages 18-65) that is ...,b'LAC Equity Lab tabulations of SEDLAC (CEDLAS...
936,9.0.Employee.B40,Employees-Bottom 40 Percent (%),Share of the labor force (ages 18-65) in the b...,b'LAC Equity Lab tabulations of SEDLAC (CEDLAS...
937,9.0.Employee.T60,Employees-Top 60 Percent (%),Share of the labor force (ages 18-65) in the t...,b'LAC Equity Lab tabulations of SEDLAC (CEDLAS...
953,9.1.Employee.All,"Employees (%), Male",Share of the male labor force (ages 18-65) tha...,b'LAC Equity Lab tabulations of SEDLAC (CEDLAS...
954,9.1.Employee.B40,"Employees-Bottom 40 Percent (%), Male",Share of the male labor force (ages 18-65) in ...,b'LAC Equity Lab tabulations of SEDLAC (CEDLAS...
...,...,...,...,...
8115,IT.MLT.EMPL,Telephone mainlines per employee,,b''
8167,IT.TEL.EMPL.TO,"Telephone employees, total",Telephone employees refer to the total full-ti...,"b'International Telecommunication Union, World..."
8176,IT.TEL.TOTL.EM,Fixed line and mobile cellular subscriptions p...,Fixed line and mobile cellular subscriptions p...,"b'International Telecommunication Union, World..."
11575,SE.PRM.BIMP.5,(Impartial Decision-Making) average score for ...,,"b'World Bank, Global Education Policy Dashboard'"


経済間の所得格差の要因を探るために，次の生産関数を考えよう。

$$
Y_i=A_iK_i^\alpha\left(h_iH_iL_i\right)^{1-\alpha}
,\quad
0<\alpha<1
$$ (eq:5-production)

* $i$：経済`i`
* $Y_i$：GDP
* $K_i$：物的資本
* $A_i$：全要素生産性（TFP）
* $H_i$：労働者一人当たりの人的資本
* $h_i$：労働者一人当たりの平均労働時間
* $L_i$：労働者数

GDPは労働，物的人的資本とTFPによって決定されると考える，ということである。このアプローチは「大雑把」な印象を否めないが，データを使う事により，重要な知見を得ることが可能となる。次に，生産関数を一人当たりに書き換えよう。

$$
\frac{Y_i}{L_i}=A_i
\left(
    \frac{K_i}{L_i}
\right)^{\alpha}
\left(
h_iH_i
\right)^{1-\alpha}
\quad\Rightarrow\quad
y_i=A_ik_i^{\alpha}\left(h_iH_i\right)^{1-\alpha}
$$ (eq:5-yi)

この式を使い経済間の一人当たりGDPの違いを考察する事になる。即ち，$A_i$，$k_i$，$h_i$，$H_i$の違いを使って$y_i$の違いを説明しようという事である。以下では，$h_iH_i$と$k_i$，$h_i$，$H_i$を合わせた項を次のように呼ぶ事にする。

$$
\begin{align*}
&\text{（労働者一人当たり）人的資本サービス： } h_iH_i\\
&\text{（労働者一人当たり）蓄積生産要素： } k_i^{\alpha}\left(h_iH_i\right)^{1-\alpha}
\end{align*}
$$


使うデータ・セットには，$y_i$，$k_i$，$h_i$に対応する変数があり，蓄積生産要素がどれだけ一人当たりGDPに寄与するかを数値で表すことができる。一方，全要素生産性は次の式で求めることができる。

$$
A_i=\dfrac{y_i}{k_i^{\alpha}\left(h_iH_i\right)^{1-\alpha}}
$$ (eq:5-Ai)

これらの式とデータを使って，一人当たりGDPに対する蓄積生産要素と全要素生産性の寄与度を数量化する事になる。

式に当てはめると次のようになる。

<!--
* $i$：経済`i`
* $Y_i$：GDP
* $K_i$：物的資本
* $A_i$：全要素生産性（TFP）
* $H_i$：労働者一人当たりの人的資本
* $h_i$：労働者一人当たりの平均労働時間
* $L_i$：労働者数 
-->

## World Bank Data Code
- GDP (PPP)
    - https://databank.worldbank.org/metadataglossary/world-development-indicators/series/NY.GDP.MKTP.PP.CD
    - https://databank.worldbank.org/metadataglossary/world-development-indicators/series/NY.GDP.MKTP.PP.KD
- Number of persons engaged
    - https://govdata360.worldbank.org/indicators/hcba778ee?country=BRA&indicator=3238&product=0&viz=line_chart&years=1995,2014

## Data Code
* 一人当たりGDP：$y_i\equiv\dfrac{Y_i}{L_i}=$ `NY.GDP.MKTP.PP.CD`/`SL.EMP.TOTL`
* 一人当たり資本：$k_i\equiv\dfrac{K_i}{L_i}=$ `ck`/`emp`
* 労働者一人当たり人的資本サービス：$h_iH_i=$ `avh`x`hc`
* 資本の所得シャア：$\alpha=1/3$を仮定

これに従ってコードを書いていこう。

In [ ]:
matches = wb.search('employee')[['id', 'name', 'sourceNote', 'sourceOrganization']]


In [1]:
%pip install pandas-datareader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 2.4 MB/s eta 0:00:00a 0:00:01
  Using cached lxml-4.9.1.tar.gz (3.4 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for lxml: filename=lxml-4.9.1-cp38-cp38-macosx_10_14_arm64.whl size=3029485 sha256=2b5c292e08114a35012309188a4245348eb9e8756b983de16688d814f644f4c5
  Stored in directory: /Users/k-kakimoto/Library/Caches/pip/wheels/03/02/29/e73e6bd0b04b9ddb5e40fe507107c1e6e148bf8521a1964685
Successfully built lxml
You should consider upgrading via the '/Users/k-kakimoto/dev/ml_samples/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
